In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Importing packages
import tensorflow as tf
import os
import cv2
import imageio
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.patches as mpatches
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.models import Model
import random as rn
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import VGG16
import datetime
import glob
import warnings
from tensorflow.keras import models, layers
from sklearn.metrics import cohen_kappa_score
import math
from keras.regularizers import l1 ,l2
import keras
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from matplotlib import pyplot as plt

import argparse
import os
import warnings

from keras.callbacks import Callback
from keras import backend as K
warnings.filterwarnings('ignore')




# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

img_path = list()
img_fullname = list()
img_name = list()
for dirname, _, filenames in os.walk('/kaggle/input/aptos2019-blindness-detection/train_images/'):
    for filename in filenames:
        img_path.append(os.path.join(dirname, filename))
        temp = os.path.join(dirname, filename)
        temp = temp.split("/")[-1]
        img_fullname.append(temp)
        temp = temp.split(".")[0]
        img_name.append(str(temp))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**1.1 Reading data**

In [ ]:
pre_df = pd.DataFrame()
pre_df['path'] = img_path
pre_df['fullname'] = img_fullname
pre_df['id_code'] = img_name

In [ ]:
# Reading DR grades files of 2019, Messidor & IDRiD
train2019 = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
train2019 = train2019[['id_code', 'diagnosis']]

In [ ]:
# Fetching DR grades of the respective images of 2019
labels = list()
id_code = list()
for i, j in train2019.iterrows():
    id_code.append(j['id_code'])
    labels.append(j['diagnosis'])

In [ ]:
train_labels = pd.DataFrame() # Storing the image file name & DR grades into the dataframe
train_labels['id_code'] = id_code
train_labels['diagnosis'] = labels

In [ ]:
# Merging the image path & DR grades 
pre_df = pd.merge(pre_df, train_labels, on='id_code', how='left')

In [ ]:
pre_df = pre_df[pre_df['diagnosis'].notna()] # removing row if the DR grade is NaN

In [ ]:
print(pre_df[pre_df['diagnosis'].isnull()]) # Verify if there is any NaN present in the DR grades field

In [ ]:
convert_dict = {'diagnosis': str}
  
pre_df = pre_df.astype(convert_dict)

**Train and test data split up**

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
y = pre_df['diagnosis'].values
X_train, X_test, y_train, y_test = train_test_split(pre_df, y, test_size=0.20, stratify=y)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
# reference URL - https://vijayabhaskar96.medium.com/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
tf.keras.backend.clear_session()
from keras_preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1./255., validation_split=0.20, rotation_range=15, fill_mode='nearest', width_shift_range=0.1, height_shift_range=0.1, 
                horizontal_flip=True, vertical_flip=True)

train_generator=datagen.flow_from_dataframe(dataframe=X_train, directory=None, x_col="path", y_col="diagnosis", subset="training", 
                batch_size=8, seed=10, shuffle=True, class_mode="categorical", drop_duplicates = False, target_size=(320,320))

valid_generator=datagen.flow_from_dataframe(dataframe=X_train, directory=None, x_col="path", y_col="diagnosis", subset="validation", 
                        batch_size=8, seed=10, shuffle=True, class_mode="categorical", drop_duplicates = False, target_size=(320,320))

test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(dataframe=X_test, directory=None, x_col="path", 
                y_col=None, batch_size=1,shuffle=False, class_mode=None, target_size=(320,320))

**Building VGG16 Model**

In [ ]:
os.environ['PYTHONHASHSEED'] = '0'

##https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
## Have to clear the session. If you are not clearing, Graph will create again and again and graph size will increses. 
## Varibles will also set to some value from before session
tf.keras.backend.clear_session()

## Set the random seed values to regenerate the model.
tf.keras.backend.clear_session()
np.random.seed(0)
rn.seed(0)

vgg_base = VGG16(include_top=False, weights=None, input_shape=(320,320,3))

# Freeze base model
vgg_base.trainable = False


#Input layer
input_layer = Input(shape=(320,320,3))

vgg_layer= vgg_base(input_layer)

#Conv Layer
Conv1 = Conv2D(128, 10, strides=1 , padding = 'valid', activation='relu')(vgg_layer)


#MaxPool Layer
Conv2 = Conv2D(64, 1, strides=1 , padding = 'valid', activation='relu')(Conv1)

flatten = Flatten(data_format='channels_last')(Conv2)
#output layer
outputs = Dense(5,activation='softmax')(flatten)


#Creating a model
vgg_model = Model(inputs=input_layer,outputs=outputs)
vgg_model.load_weights('../input/vgg-pretrained-weights/weights-vgg.hdf5')

#creating object to ReduceLROnPlateau class to decay lr by 10% If validation accuracy is less than previous epoch accuracy(cond1)
reducelrate = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.96, verbose=1, patience=2, cooldown=2)

callbacks_list = [reducelrate] #Merging all the callbacks in a list

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

vgg_model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy']) #Compiling model with cross-entropy as loss
print(vgg_model.summary())


**Training the VGG16 model**

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
vgg_model.fit_generator(generator=train_generator, steps_per_epoch=STEP_SIZE_TRAIN, validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID, epochs=80, callbacks=callbacks_list)

**Prediction on Test images**

In [ ]:
y_pred = vgg_model.predict(test_generator)
y_pred_class = np.argmax(y_pred, axis=1)

**Calculating Quadratic cohen kappa score**

In [ ]:
cohen = cohen_kappa_score(y_pred_class, y_test.astype('int'), weights='quadratic')
print("Quadratic Cohen kappa score of VGG model on test data is - %.3f" %cohen)

**Calculating Accuracy & Macro F1 score on test data**

In [ ]:
print ('Classification Report : \n', classification_report(y_test.astype('int'), y_pred_class))
sns.heatmap(confusion_matrix(y_test.astype('int'), y_pred_class), annot=True, fmt="d");
plt.title("Confusion matrix")
plt.ylabel('Actual class')
plt.xlabel('Predicted class')
plt.show()

**Reading Test data**

In [ ]:
img_path = list()
img_fullname = list()
img_name = list()
for dirname, _, filenames in os.walk('../input/aptos2019-blindness-detection/test_images/'):
    for filename in filenames:
        img_path.append(os.path.join(dirname, filename))
        temp = os.path.join(dirname, filename)
        temp = temp.split("/")[-1]
        img_fullname.append(temp)
        temp = temp.split(".")[0]
        img_name.append(str(temp))


In [ ]:
pre_df = pd.DataFrame()
pre_df['path'] = img_path
pre_df['fullname'] = img_fullname
pre_df['id_code'] = img_name

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator_2019=test_datagen.flow_from_dataframe(dataframe=pre_df, directory=None, x_col="path", 
                y_col=None, batch_size=1, shuffle=False, class_mode=None, target_size=(320,320))

In [ ]:
y_pred_2019 = vgg_model.predict(test_generator_2019)
y_pred_class_2019 = np.argmax(y_pred_2019, axis=1)

In [ ]:
filenames = list()
for i in test_generator_2019.filenames:
    temp = i.split('.')[-2]
    temp = temp.split('/')[-1]
    filenames.append(temp)

In [ ]:
submission_vgg = pd.DataFrame()
submission_vgg['id_code'] = filenames
submission_vgg['diagnosis'] = y_pred_class_2019

In [ ]:
submission_vgg

In [ ]:
submission_vgg.to_csv('./submission.csv', index=False)